# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages
import numpy as np
import pandas as pd
import cassandra
import glob
import json
import csv
import os
import re

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# For every filepath in the file path list
for f in file_path_list:
    # Reading csv file
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        # Creating a csv reader object
        csvreader = csv.reader(csvfile)
        next(csvreader)

        # Extract each data row one-by-one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line)

# Uncomment the code below if you would like to get the total number of rows 
# print(len(full_data_rows_list))

# Uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# Creating a smaller event data csv file called `event_datafile_new.csv` that will be used to \
# insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', \
                     'length', 'level', 'location', 'sessionId', 'song', 'userId'])

    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating tables usingApache Cassandra

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### QUERY-1
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4.

`SELECT artist, song, length FROM song_details WHERE sessionId = 338 AND itemInSession = 4;`

#### CREATE Statement

In [8]:
## Create Statement
create_query_1 = """
        CREATE TABLE IF NOT EXISTS song_details (
        sessionId INT, itemInSession INT, 
        artist TEXT, song TEXT, length FLOAT, 
        PRIMARY KEY (sessionId, itemInSession))
        """
try:
    session.execute(create_query_1)
except Exception as e:
    print(e)

#### Insert Statement

COLUMNS: -
`artist`: 0, `firstName`: 1, `gender`: 2,
`itemInSession`: 3, `lastName`: 4, `length`: 5,
`level`: 6, `location`: 7, `sessionId`: 8, `song`: 9, `userId`: 10

In [9]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        ## Assign the INSERT statement
        insert_query_1 = """
        INSERT INTO song_details (sessionId, itemInSession, artist, song, length)
        VALUES (%s, %s, %s, %s, %s)
        """
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(insert_query_1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

# Learnings: 
- Encountered a `SyntaxException`.

  SOLUTION: - Don't forget to *convert the data types* after reading from the CSV file.

#### SELECT Statement

In [10]:
## Add in the SELECT statement to verify the data was entered into the table correctly
select_query_1 = "SELECT artist, song, length FROM song_details WHERE sessionId = %s AND itemInSession = %s"
try:
    rows = session.execute(select_query_1, (338, 4))
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, "|", row.song, "|", row.length)

Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875


### QUERY-2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

`SELECT artist, song, firstName, lastName FROM listening_history WHERE userId = 10 AND sessionId = 182;`

#### CREATE Statement

In [11]:
## Create Statement
create_query_2 = """
        CREATE TABLE IF NOT EXISTS listening_history (
        userId INT, sessionId INT, itemInSession INT, 
        artist TEXT, song TEXT, firstName TEXT, lastName TEXT, 
        PRIMARY KEY (userId, sessionId, itemInSession))
        """
try:
    session.execute(create_query_2)
except Exception as e:
    print(e)

#### INSERT Statement

COLUMNS: -
`artist`: 0, `firstName`: 1, `gender`: 2,
`itemInSession`: 3, `lastName`: 4, `length`: 5,
`level`: 6, `location`: 7, `sessionId`: 8, `song`: 9, `userId`: 10

In [12]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        ## Assign the INSERT statement
        insert_query_2 = """
        INSERT INTO listening_history (userId, sessionId, itemInSession, artist, song, firstName, lastName)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(insert_query_2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


#### SELECT Statement

In [13]:
## Add in the SELECT statement to verify the data was entered into the table correctly
select_query_2 = "SELECT artist, song, firstName, lastName FROM listening_history WHERE userId = %s AND sessionId = %s"
try:
    rows = session.execute(select_query_2, (10, 182))
except Exception as e:
    print(e)

# From Knowledge: https://knowledge.udacity.com/questions/45076
for row in rows:
    print(row.artist, "|", row.song, "|", row.firstname, "|", row.lastname)


Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


### Query-3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

`SELECT firstName, lastName FROM user_history WHERE song = "All Hands Against His Own";`

#### CREATE Statement

# Learnings: 
Some examples of primary key definition are:

- `PRIMARY KEY (a)` :  `a` is the single partition key and there are no clustering columns.

- `PRIMARY KEY (a, b, c)` : `a` is the single partition key and `b` and `c` are the clustering columns.

- `PRIMARY KEY ((a, b), c)` : `a` and `b` compose the *composite partition key* and `c` is the clustering column.

&rarr; The primary key uniquely identifies a row in the table, as described above. A consequence of this uniqueness is that if another row is inserted using the same primary key, then an `UPSERT` occurs and an existing row with the same primary key is **replaced**. Columns that are not part of the primary key cannot define uniqueness.

In [14]:
## Create Statement
# Useful resource : https://cassandra.apache.org/doc/latest/cassandra/cql/ddl.html#primary-key
create_query_3 = """
        CREATE TABLE IF NOT EXISTS user_history (
        song TEXT, userId INT, firstName TEXT, lastName TEXT, 
        PRIMARY KEY ((song), userId))
        """
try:
    session.execute(create_query_3)
except Exception as e:
    print(e)

#### INSERT Statement

COLUMNS: -
`artist`: 0, `firstName`: 1, `gender`: 2,
`itemInSession`: 3, `lastName`: 4, `length`: 5,
`level`: 6, `location`: 7, `sessionId`: 8, `song`: 9, `userId`: 10

In [15]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        ## Assign the INSERT statement
        insert_query_3 = """
        INSERT INTO user_history (song, userId, firstName, lastName)
        VALUES (%s, %s, %s, %s)
        """
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(insert_query_3, (line[9], int(line[10]), line[1], line[4]))

#### SELECT Statement

In [16]:
## Add in the SELECT statement to verify the data was entered into the table correctly
select_query_3 = "SELECT firstName, lastName FROM user_history WHERE song = %s"
try:
    # NOTE: Take a note of the comma. Else, we will get "not all arguments converted during string formatting" warning
    rows = session.execute(select_query_3, ("All Hands Against His Own",))
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, "|", row.lastname)

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


### Drop the tables before closing out the sessions

In [17]:
drop_query_1 = "DROP TABLE IF EXISTS song_details"
try:
    rows = session.execute(drop_query_1)
except Exception as e:
    print(e)

drop_query_2 = "DROP TABLE IF EXISTS listening_history"
try:
    rows = session.execute(drop_query_2)
except Exception as e:
    print(e)

drop_query_3 = "DROP TABLE IF EXISTS user_history"
try:
    rows = session.execute(drop_query_3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()